In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("C:/Users/LENOVO/Desktop/Telco_Cust_Churn.csv")

In [3]:
print(data.columns)

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [4]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'] = data['TotalCharges'].fillna(data['TotalCharges']. median())

##kind of encoding is done
data['Churn'] = data['Churn'].map({'Yes': 1, 'No':0 })

#onehot encoding 
#assignning the numeric value for each feature set
data = pd.get_dummies(data, columns=['Contract', 'PaymentMethod'], drop_first=True)

#standadizing 
#NORMALIZING
scaler = MinMaxScaler()
data[['MonthlyCharges', 'TotalCharges']] = scaler.fit_transform(data[['MonthlyCharges', 'TotalCharges']])

#split the data into feature and target
X = data.drop('Churn', axis = 1)
y = data['Churn']